In [1]:
from transliterate import translit
import sqlite3
import pickle
import sys
import re

In [33]:
class DBaseRusNLP():
    tables = {
        "author": {
            "id": "INT PRIMARY KEY",
            "name": "text",
            "email": "text",
            "affiliation": "text"
        },

        "article": {
            "id": "INT PRIMARY KEY",
            "title": "text",
            "keywords": "text",
            "abstract": "text",
            "bibliography": "text",
            "text": "text"
        },

        'catalogue': {
            "id": "INT PRIMARY KEY",
            "author_id": "INT",
            "article_id": "INT",
            "conference_id": "INT"
        },

        'conference': {
            "id": "INT PRIMARY KEY",
            "conference": "text",
            "year": "int"
        }
    }

    def __init__(self):
        self.conn = sqlite3.connect("rus_nlp.db")
        self.cursor = self.conn.cursor()
        for name, data in self.tables.items():
            self.create_table(name, data)
        self.author_columns = self.select_columns_name("author")
        self.article_columns = self.select_columns_name("article")
        self.catalogue_columns = self.select_columns_name("catalogue")
        self.conference_columns = self.select_columns_name("conference")

    def close(self):
        self.conn.close()

    # ********************************************************************

    def create_table(self, name, columns_dict):
        query = "CREATE TABLE IF NOT EXISTS " + name + " ("
        columns = ', '.join([key + " " + val for key, val in columns_dict.items()])
        last = ")"
        self.cursor.execute(query + columns + last)
        self.conn.commit()

    # ********************************************************************

    def __update(self, table, column, value, what, condition):
        query = '''UPDATE '''+table
        set_col = ''' SET ''' + column + '''= '''+ self.__check(value)
        where = ''' WHERE ''' + what +''' = ''' + self.__check(condition)
        try:
            self.cursor.execute(query + set_col + where)
            self.conn.commit()
        except Exception as e:
            print(query + set_col + where)

    # ********************************************************************

    def delete(self, table=None, column=None, condition=None):
        pass

    # ********************************************************************

    def select(self, what, where, condition=None):
        query = "SELECT " + what + " FROM " + where
        if condition!=None:
            query += " WHERE " + condition
        self.cursor.execute(query)
        print(query)
        return self.cursor.fetchall()

    def select_max(self, table):
        query = """SELECT MAX(id) FROM """ + table + ";"
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        return rows[0][0] if rows[0][0] != None else 0
    
    def select_columns_name(self, table_name):
        query = "SELECT * FROM " + table_name
        self.cursor.execute(query)
        return [member[0] for member in self.cursor.description]

    # ********************************************************************

    def __insert(self, table, value, columns):
        query = '''INSERT INTO ''' + table
        query += ''' (''' + ''', '''.join(columns) + ''') '''
        values = ''' VALUES(''' + ''', '''.join([self.__check(i) for i in value]) + '''); '''
        self.cursor.execute(query + values)
        self.conn.commit()
        print('loaded to '+ str(table))        

    def __check(self, value):
        if type(value) == int:
            return str(value)
        elif type(value) == str:
            value = value.replace("'",""" """).replace('"', """ """).replace('\n', ' ')
            value = ''.join([x for x in value if ord(x)])
            return u''+'''"''' + value + '''"'''
        elif value == None:
            return '''NULL'''
        else:
            print(type(value))
            print(value)
            raise TypeError("Unknown type")

    # --------------------------------------------------------------------
    # --------------------------------------------------------------------

    def insert_into_conference(self, values):
        max_id = self.select_max('conference')
        for i in range(len(values)):
            row = [max_id + 1] + list(values[i])
            self.__insert('conference', row, columns=['id', 'conference', 'year'])

    # --------------------------------------------------------------------
    # --------------------------------------------------------------------

    def insert_into_author(self, id_, name, affiliation, email):
        self.__insert('author', (id_, name, email, affiliation), ["id", "name", "email", "affiliation"])

    # --------------------------------------------------------------------
    # --------------------------------------------------------------------

    def insert_into_article(self, id_, title, text, abstract, bibliography, keywords):
        self.__insert('article', (id_, title, keywords, abstract, bibliography, text), 
                     ['id', 'title', 'keywords', 'abstract', 'bibliography', 'text'])

    # --------------------------------------------------------------------
    #    INSERT NEW ARTICLE
    # --------------------------------------------------------------------

    def insert_new_article(self, author, article, conference_name, year):
        try:
            article_id = self.get_article_id(article)
        except Exception as e:
            print(e)
        author_id = self.get_author_id(author)
        conference_id = self.get_conference_id(conference_name, year)
        for auth_id in author_id:
            self.insert_to_catalogue(auth_id, article_id, conference_id)
            
    # --------------------------------------------------------------------
    # --------------------------------------------------------------------

    def get_author_id(self, author):
        list_of_authors_id = []
        for auth in author:
            author_id = self.select_id_from_author(auth['name'])
            if not author_id:
                author_id = self.select_max('author') + 1
                if type(auth['email'])!=str:
                    print(str(auth['email']) +" "+ str(type(auth['email'])))
                    raise Exception("2 emails")
                self.insert_into_author(id_=author_id, name=auth['name'], 
                                    email=auth['email'],
                                    affiliation=auth['affiliation'])
            list_of_authors_id.append(author_id)
        return list_of_authors_id

    # --------------------------------------------------------------------

    def select_id_from_author(self, name):
        where = ' name = ' + self.__check(name)
        rows = self.select(what='id', where='author', condition=where)
        return rows[0][0] if rows != [] else None

    # --------------------------------------------------------------------
    # --------------------------------------------------------------------

    def get_article_id(self, article):
        article_id = self.select_id_from_article(article['title'])
        print(article['title']+' id ='+ str(article_id))
        if not article_id:
            article_id = self.select_max('article') + 1
            if 'bibliography' not in article.keys():
                article['bibliography']=None
            else:
                print(article['bibliography'])
                raise Exception("here, bibliography")
            if 'keywords' not in article.keys():
                article['keywords']=None
            if type(article['text']==list):
                article['text']=' '.join(article['text'])
            else:
                print('wrong type')
                print(type(article['text']))
            self.insert_into_article(id_=article_id, title=article['title'], text=article['text'],
                                     abstract=article['abstract'], bibliography=article['bibliography'],
                                     keywords=article['keywords'])
        return article_id
    

    # --------------------------------------------------------------------

    def select_id_from_article(self, title):
        where = """title=""" + self.__check(title)
        rows = self.select(what='id', where='article', condition=where)
        print(rows)
        return rows[0][0] if rows != [] else None

    # --------------------------------------------------------------------
    # --------------------------------------------------------------------

    def get_conference_id(self, conference_name, year):
        conference_id = self.select_id_from_conference(conference_name, year)
        if not conference_id:
            print(conference_name+ " "+ str(year))
            raise ValueError('wrong combination of year and conference name')
        return conference_id

    def select_id_from_conference(self, conference_name, year):
        where = ''' year = ''' + str(year) + ''' AND conference = "''' + conference_name + '";'
        rows = self.select(what='id', where='conference', condition=where)
        return rows[0][0] if rows != [] else None
    
    
    # --------------------------------------------------------------------
    # --------------------------------------------------------------------
    
    def insert_to_catalogue(self, auth_id, article_id, conference_id):
        catalog_id = self.select_id_from_catalogue(auth_id, article_id, conference_id)
        if not catalog_id:
            catalog_id = self.select_max('catalogue') + 1
            self.__insert('catalogue', (int(catalog_id), int(auth_id),
                                  int(article_id), int(conference_id)), 
                          columns=['id', 'author_id', 'article_id', 'conference_id'])
    
    def select_id_from_catalogue(self, auth_id, article_id, conference_id):
        where = ''' author_id = ''' + str(auth_id) + ''' AND article_id = ''' + str(article_id) + ''' AND conference_id = ''' + str(conference_id) + ';'
        rows = self.select(what='id', where='catalogue', condition=where)
        return rows[0][0] if rows != [] else None
        

    # ********************************************************************
    
    def select_articles_of_author(self, name):
        li_name = [translit(name, 'ru'), translit(name, 'ru', reversed=True)]
        what = "DISTINCT article.title"
        where = "catalogue JOIN author JOIN article ON catalogue.author_id=author.id AND catalogue.article_id=article.id"
        condition = "author.name LIKE '%"+self.__check(li_name[0])[1:-1]+"%' OR author.name LIKE '%"+self.__check(li_name[1])[1:-1]+"%'"
        result = self.select(what, where, condition)
        return result
    
    
    def select_articles_from_conference(self, conf_name, year=None):
        what = "DISTINCT article.title"
        where = "catalogue JOIN conference JOIN article ON catalogue.conference_id=conference.id AND catalogue.article_id=article.id"
        condition = "conference.conference='"+conf_name+"'"
        if year != None:
            condition += "AND conference.year="+str(year)
        result = self.select(what, where, condition)
        return result
    
    def select_article_by_affiliation(self, affiliation):
        li_name = [self.__check(translit(affiliation, 'ru'))[1:-1], 
                   self.__check(translit(affiliation, 'ru', reversed=True))[1:-1]]
        what = "DISTINCT article.title"
        where = "catalogue JOIN author JOIN article ON catalogue.author_id=author.id AND catalogue.article_id=article.id"
        condition = "author.affiliation LIKE '%"+li_name[0]+"%' OR author.affiliation LIKE '%"+li_name[1]+"%'"
        result = self.select(what, where, condition)
        return result
    
    # ********************************************************************

    def load_to_dialog(self, dict_with_data):
        self.load('Dialog', dict_with_data)

    def load_to_aist(self, dict_with_data):
        self.load('AIST', dict_with_data)

    def load_to_ainl(self, dict_with_data):
        self.load('AINL', dict_with_data)

    def load_to_russir(self, dict_with_data):
        self.load('RuSSIR', dict_with_data)

    # --------------------------------------------------------------------

    def load(self, name, dict_with_data):
        counter = 0
        for article in dict_with_data:
            self.insert_new_article(author=article['author'], 
                                    article=article['text'],
                                    conference_name=name, 
                                    year=int(article['year']))
            counter += 1

            
    


Подключаемся к БД:

In [48]:
c = DBaseRusNLP()

Добавляем (в пустую БД) конференции:

In [49]:
try:
    for i in range(18):
        c.insert_into_conference([('Dialog',2000+i)])
    for i in range(15, 18):
        c.insert_into_conference([('AINL',2000+i)])
    for i in range(7,18):
        c.insert_into_conference([('AIST',2000+i)])
    for i in range(8,18):
        c.insert_into_conference([('RuSSIR',2000+i)])
            #c.insert_into_author()
except Exception as e:
    print(e)

loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference
loaded to conference


Закрываем БД:

In [47]:
c.close()

Обычные запросы напрямую к БД:

In [36]:
c.cursor.execute("""SELECT * FROM conference""")
d = c.cursor.fetchall()
print(d)

[(1, 'Dialog', 2000), (2, 'Dialog', 2001), (3, 'Dialog', 2002), (4, 'Dialog', 2003), (5, 'Dialog', 2004), (6, 'Dialog', 2005), (7, 'Dialog', 2006), (8, 'Dialog', 2007), (9, 'Dialog', 2008), (10, 'Dialog', 2009), (11, 'Dialog', 2010), (12, 'Dialog', 2011), (13, 'Dialog', 2012), (14, 'Dialog', 2013), (15, 'Dialog', 2014), (16, 'Dialog', 2015), (17, 'Dialog', 2016), (18, 'Dialog', 2017), (19, 'AINL', 2015), (20, 'AINL', 2016), (21, 'AINL', 2017), (22, 'AIST', 2007), (23, 'AIST', 2008), (24, 'AIST', 2009), (25, 'AIST', 2010), (26, 'AIST', 2011), (27, 'AIST', 2012), (28, 'AIST', 2013), (29, 'AIST', 2014), (30, 'AIST', 2015), (31, 'AIST', 2016), (32, 'RuSSIR', 2008), (33, 'RuSSIR', 2009), (34, 'RuSSIR', 2010), (35, 'RuSSIR', 2011), (36, 'RuSSIR', 2012), (37, 'RuSSIR', 2013), (38, 'RuSSIR', 2014), (39, 'RuSSIR', 2015), (40, 'RuSSIR', 2016), (41, 'RuSSIR', 2017)]


Общая информация по БД:

In [59]:
print(c.select_max('article'))
print(c.select_max('conference'))
print(c.select_max('author'))
print(c.select_max('catalogue'))

1020
42
1546
2176


Загрузка статей AINL:

In [57]:
with open("ainl.pickle", 'rb') as f:
    dialog = pickle.load(f)
    print(len(dialog), 'articles')
    c.load_to_ainl(dialog)

74 articles
SELECT id FROM article WHERE title="Multi-Representation Approach to Text Regression of Financial Risks"
[(89,)]
Multi-Representation Approach to Text Regression of Financial Risks id =89
SELECT id FROM author WHERE  name = "Roman Trusov"
SELECT id FROM author WHERE  name = "Sergey Ovcharenko"
SELECT id FROM author WHERE  name = "Alexey Natekin"
SELECT id FROM author WHERE  name = "Alois Knoll"
SELECT id FROM author WHERE  name = "Pavel Kalaidin"
SELECT id FROM author WHERE  name = "Aida Fazylova"
SELECT id FROM conference WHERE  year = 2015 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 169 AND article_id = 89 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 170 AND article_id = 89 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 171 AND article_id = 89 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 172 AND article_id = 89 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 173

SELECT id FROM conference WHERE  year = 2015 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 212 AND article_id = 104 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 213 AND article_id = 104 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 214 AND article_id = 104 AND conference_id = 19;
SELECT id FROM article WHERE title="Comparison of Sentence Similarity Measures as Features for Russian Paraphrase Classification "
[(105,)]
Comparison of Sentence Similarity Measures as Features for Russian Paraphrase Classification  id =105
SELECT id FROM author WHERE  name = "Ekaterina Pronoza"
SELECT id FROM author WHERE  name = "Elena Yagunova "
SELECT id FROM conference WHERE  year = 2015 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 137 AND article_id = 105 AND conference_id = 19;
SELECT id FROM catalogue WHERE  author_id = 215 AND article_id = 105 AND conference_id = 19;
SELECT id FROM article WHERE title="A Study

Using Linguistic Activity In Social Networks To Predict And Interpret Dark Psychological Traits id =118
SELECT id FROM author WHERE  name = "Arseny Moskvichev"
SELECT id FROM author WHERE  name = "Marina Dubova"
SELECT id FROM author WHERE  name = "Sergey Menshov"
SELECT id FROM author WHERE  name = "Andrey Filchenkov"
SELECT id FROM conference WHERE  year = 2017 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 203 AND article_id = 118 AND conference_id = 21;
SELECT id FROM catalogue WHERE  author_id = 244 AND article_id = 118 AND conference_id = 21;
SELECT id FROM catalogue WHERE  author_id = 245 AND article_id = 118 AND conference_id = 21;
SELECT id FROM catalogue WHERE  author_id = 222 AND article_id = 118 AND conference_id = 21;
SELECT id FROM article WHERE title="Combined Feature Representation for Emotion Classification from Russian Speech "
[(119,)]
Combined Feature Representation for Emotion Classification from Russian Speech  id =119
SELECT id FROM author W

SELECT id FROM conference WHERE  year = 2017 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 274 AND article_id = 129 AND conference_id = 21;
SELECT id FROM catalogue WHERE  author_id = 275 AND article_id = 129 AND conference_id = 21;
SELECT id FROM article WHERE title="Semantic Feature Aggregation for Gender Identiﬁcation in Russian Facebook"
[(130,)]
Semantic Feature Aggregation for Gender Identiﬁcation in Russian Facebook id =130
SELECT id FROM author WHERE  name = "Polina Panicheva"
SELECT id FROM author WHERE  name = "Aliia Mirzagitova"
SELECT id FROM author WHERE  name = "Yanina Ledovaya"
SELECT id FROM conference WHERE  year = 2017 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 116 AND article_id = 130 AND conference_id = 21;
SELECT id FROM catalogue WHERE  author_id = 276 AND article_id = 130 AND conference_id = 21;
SELECT id FROM catalogue WHERE  author_id = 205 AND article_id = 130 AND conference_id = 21;
SELECT id FROM article WHERE

An Adaptive Web Interface for Microtask-Based Crowdsourcing id =144
SELECT id FROM author WHERE  name = "Ilya Sukhopluev"
SELECT id FROM author WHERE  name = "Dmitry Ustalov"
SELECT id FROM conference WHERE  year = 2016 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 307 AND article_id = 144 AND conference_id = 20;
SELECT id FROM catalogue WHERE  author_id = 63 AND article_id = 144 AND conference_id = 20;
SELECT id FROM article WHERE title="Towards cluster validity index evaluation and selection"
[(145,)]
Towards cluster validity index evaluation and selection id =145
SELECT id FROM author WHERE  name = "Andrey Filchenkov"
SELECT id FROM author WHERE  name = "Sergey Muravyov"
SELECT id FROM author WHERE  name = "Vladimir Parfenov"
SELECT id FROM conference WHERE  year = 2016 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 222 AND article_id = 145 AND conference_id = 20;
SELECT id FROM catalogue WHERE  author_id = 308 AND article_id = 145 AND co

SELECT id FROM catalogue WHERE  author_id = 328 AND article_id = 159 AND conference_id = 20;
SELECT id FROM catalogue WHERE  author_id = 329 AND article_id = 159 AND conference_id = 20;
SELECT id FROM catalogue WHERE  author_id = 330 AND article_id = 159 AND conference_id = 20;
SELECT id FROM article WHERE title="Measuring influencers in Twitter ad-hoc discussions: Active users vs. internal networks in the discourse on Biryuliovo bashings in 2013"
[(160,)]
Measuring influencers in Twitter ad-hoc discussions: Active users vs. internal networks in the discourse on Biryuliovo bashings in 2013 id =160
SELECT id FROM author WHERE  name = "Svetlana S. Bodrunova "
SELECT id FROM author WHERE  name = "Ivan S. Blekanov "
SELECT id FROM author WHERE  name = "Alexey Maksimov "
SELECT id FROM conference WHERE  year = 2016 AND conference = "AINL";
SELECT id FROM catalogue WHERE  author_id = 331 AND article_id = 160 AND conference_id = 20;
SELECT id FROM catalogue WHERE  author_id = 332 AND article_

Загрузка статей Dialog:

In [58]:
with open("dialogue.pickle", 'rb') as f:
    dialog = pickle.load(f)
    print(len(dialog), 'articles')
    c.load_to_dialog(dialog)

868 articles
SELECT id FROM article WHERE title="ИСПОЛЬЗОВАНИЕ МАРКЕРОВ АКТАНТНЫХ ПОЗИЦИЙ ПРИ АНАЛИЗЕ ДЕЛОВЫХ ТЕКСТОВ ДЛЯ РАСШИРЕНИЯ ЛОГИЧЕСКОЙ СХЕМЫ ПРЕДМЕТНОЙ ОБЛАСТИ (THE CONTEXT SCHEMA OF PREDICATE ARGUMENTS FOR AUTOMATIC EXPANSION OF A DOMAIN ONTOLOGY)"
[]
ИСПОЛЬЗОВАНИЕ МАРКЕРОВ АКТАНТНЫХ ПОЗИЦИЙ ПРИ АНАЛИЗЕ ДЕЛОВЫХ ТЕКСТОВ ДЛЯ РАСШИРЕНИЯ ЛОГИЧЕСКОЙ СХЕМЫ ПРЕДМЕТНОЙ ОБЛАСТИ (THE CONTEXT SCHEMA OF PREDICATE ARGUMENTS FOR AUTOMATIC EXPANSION OF A DOMAIN ONTOLOGY) id =None
loaded to article
SELECT id FROM author WHERE  name = "Азарова И.В. "
loaded to author
SELECT id FROM author WHERE  name = "Гребеньков А.С. "
loaded to author
SELECT id FROM author WHERE  name = "Ландо Т.М. "
loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 334 AND article_id = 161 AND conference_id = 9;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 335 AND article_id = 161 AND conference_id = 9;
loaded to catalogu

loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 353 AND article_id = 171 AND conference_id = 9;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 354 AND article_id = 171 AND conference_id = 9;
loaded to catalogue
SELECT id FROM article WHERE title="АВТОМАТИЗАЦИЯ ОНТОЛОГИЧЕСКОГО ИНЖИНИРИНГА В СИСТЕМАХ ИЗВЛЕЧЕНИЯ ЗНАНИЙ ИЗ ТЕКСТА (AUTOMATIZATION OF AN ONTHOLOGICAL ENGINEERING FOR SYSTEMS OF KNOWLEDGE MINING IN TEXT)"
[]
АВТОМАТИЗАЦИЯ ОНТОЛОГИЧЕСКОГО ИНЖИНИРИНГА В СИСТЕМАХ ИЗВЛЕЧЕНИЯ ЗНАНИЙ ИЗ ТЕКСТА (AUTOMATIZATION OF AN ONTHOLOGICAL ENGINEERING FOR SYSTEMS OF KNOWLEDGE MINING IN TEXT) id =None
loaded to article
SELECT id FROM author WHERE  name = "Ермаков А.Е. "
loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 355 AND article_id = 172 AND conference_id = 9;
loaded to catalogue
SELECT id FROM article WHERE 

loaded to catalogue
SELECT id FROM article WHERE title="ПРОТИВ «РАЗЛОЖЕНИЯ СМЫСЛА»: УЗНАВАНИЕ В СЕМАНТИКЕ ИДИОМ (AGAINST DECOMPOSITION OF MEANING: RECOGNITION IN SEMANTICS OF IDIOMS)"
[]
ПРОТИВ «РАЗЛОЖЕНИЯ СМЫСЛА»: УЗНАВАНИЕ В СЕМАНТИКЕ ИДИОМ (AGAINST DECOMPOSITION OF MEANING:
RECOGNITION IN SEMANTICS OF IDIOMS) id =None
loaded to article
SELECT id FROM author WHERE  name = "Баранов А.Н. "
loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 372 AND article_id = 183 AND conference_id = 9;
loaded to catalogue
SELECT id FROM article WHERE title="ЭВОЛЮЦИЯ ФОРМ РЕЧЕВОГО АРГУМЕНТАТИВНОГО ПОВЕДЕНИЯ КАК ОДИН ИЗ АСПЕКТОВ СТАНОВЛЕНИЯ КОММУНИКАТИВНОЙ КОМПЕТЕНЦИИ ЯЗЫКОВОЙ ЛИЧНОСТИ (EVOLUTION OF ARGUMENTATIVE LANGUAGE BEHAVIOUR PATTERNS AS AN ASPECT OF COMMUNICATIVE COMPETENCE GENESIS)"
[]
ЭВОЛЮЦИЯ ФОРМ РЕЧЕВОГО АРГУМЕНТАТИВНОГО ПОВЕДЕНИЯ КАК ОДИН ИЗ АСПЕКТОВ СТАНОВЛЕНИЯ КОММУНИКАТИВНОЙ КОМПЕТЕНЦИИ ЯЗЫКОВОЙ ЛИЧНОСТИ (E

SELECT id FROM catalogue WHERE  author_id = 387 AND article_id = 194 AND conference_id = 9;
loaded to catalogue
SELECT id FROM article WHERE title="СИНКРЕТИЗМ ЧАСТЕЙ РЕЧИ В РУССКОМ И УРАЛЬСКИХ ЯЗЫКАХ (К ВОПРОСУ О СПЕЦИФИКЕ СОСТАВЛЕНИЯ ДВУЯЗЫЧНЫХ СЛОВАРЕЙ ДЛЯ ЯЗЫКОВ РАЗНЫХ ТИПОВ) (PARTS OF SPEECH SYNCRETISM IN RUSSIAN AND URALIC (THE PROBLEM OF THE COMPOSITION OF BILINGUAL DICTIONARIES FOR INFLECTIONAL AND AGGLUTINATIVE LANGUAGES))"
[]
СИНКРЕТИЗМ ЧАСТЕЙ РЕЧИ В РУССКОМ И УРАЛЬСКИХ ЯЗЫКАХ (К ВОПРОСУ О СПЕЦИФИКЕ СОСТАВЛЕНИЯ ДВУЯЗЫЧНЫХ СЛОВАРЕЙ ДЛЯ ЯЗЫКОВ РАЗНЫХ ТИПОВ) (PARTS OF SPEECH SYNCRETISM IN RUSSIAN AND URALIC (THE PROBLEM OF THE COMPOSITION OF BILINGUAL DICTIONARIES FOR INFLECTIONAL AND AGGLUTINATIVE LANGUAGES)) id =None
loaded to article
SELECT id FROM author WHERE  name = "Кузнецова А.И. "
loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 388 AND article_id = 195 AND conference_id = 9;
loaded to ca

loaded to article
SELECT id FROM author WHERE  name = "Маркасова Е.В. "
loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 406 AND article_id = 206 AND conference_id = 9;
loaded to catalogue
SELECT id FROM article WHERE title="СИНТАКСИС КОРРЕЛЯТИВНЫХ КОНСТРУКЦИЙ РУССКОГО ЯЗЫКА С ПОЗИЦИИ ГЕНЕРАТИВНОЙ ГРАММАТИКИ (SYNTAX OF CORRELATIVE CONSTRUCTIONS IN RUSSIAN: А GENERATIVE APPROACH)"
[]
СИНТАКСИС КОРРЕЛЯТИВНЫХ КОНСТРУКЦИЙ РУССКОГО ЯЗЫКА С ПОЗИЦИИ ГЕНЕРАТИВНОЙ ГРАММАТИКИ (SYNTAX OF CORRELATIVE CONSTRUCTIONS IN RUSSIAN: А GENERATIVE APPROACH) id =None
loaded to article
SELECT id FROM author WHERE  name = "Митренина О.В. "
loaded to author
SELECT id FROM conference WHERE  year = 2008 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 407 AND article_id = 207 AND conference_id = 9;
loaded to catalogue
SELECT id FROM article WHERE title="КОРПУСНОЕ ИССЛЕДОВАНИЕ СОЧЕТАЕМОСТНЫХ ПРЕДПОЧТЕНИЙ ЧАСТ

...

loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1352 AND article_id = 970 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="Semantic Role labeling with neuRal netwoRkS foR textS in RuSSian"
[]
Semantic Role labeling with neuRal netwoRkS foR textS in RuSSian id =None
loaded to article
SELECT id FROM author WHERE  name = "Shelmanov A. O. "
loaded to author
SELECT id FROM author WHERE  name = "Devyatkin D. A. "
loaded to author
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1470 AND article_id = 971 AND conference_id = 18;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1471 AND article_id = 971 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="РевеРсивный пеРевод как инстРумент лингвистического анализа дискуРсивных слов"
[]
РевеРсивный пеРевод как инстРумент лингвистического анализа дискуРсивных слов id =None
loaded to article
SE

loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1492 AND article_id = 979 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="CoreferenCe resolution in russian: state-of-the-art approaChes appliCation and evolvement"
[]
CoreferenCe resolution in russian: state-of-the-art approaChes appliCation and evolvement id =None
loaded to article
SELECT id FROM author WHERE  name = "Sysoev A. A. "
loaded to author
SELECT id FROM author WHERE  name = "Andrianov I. A. "
loaded to author
SELECT id FROM author WHERE  name = "Khadzhiiskaia A. Y. "
loaded to author
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1493 AND article_id = 980 AND conference_id = 18;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1494 AND article_id = 980 AND conference_id = 18;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1495 AND article_id = 980 AND conference_id = 18;
loaded t

loaded to article
SELECT id FROM author WHERE  name = "Arefyev N. V. "
loaded to author
SELECT id FROM author WHERE  name = "Ermolaev P. A. "
loaded to author
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1504 AND article_id = 993 AND conference_id = 18;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1505 AND article_id = 993 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="Semantic claSSification of RuSSian HeteRonominative noun PHRaSeS on tHe mateRial of RucoR coRPuS"
[]
Semantic claSSification of RuSSian HeteRonominative noun PHRaSeS on tHe mateRial of RucoR coRPuS id =None
loaded to article
SELECT id FROM author WHERE  name = "Azerkovich I. L. "
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1125 AND article_id = 994 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="Ав

SELECT id FROM author WHERE  name = "Шурупова Т. Ю. "
loaded to author
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1519 AND article_id = 1007 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="СтатиСтичеСкий анализ результатов перцептивного оценивания глубины проСодичеСких швов  в руССком звучащем текСте"
[]
СтатиСтичеСкий анализ результатов перцептивного оценивания глубины проСодичеСких швов  в руССком звучащем текСте id =None
loaded to article
SELECT id FROM author WHERE  name = "Смирнова О. С. "
loaded to author
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1520 AND article_id = 1008 AND conference_id = 18;
loaded to catalogue
SELECT id FROM article WHERE title="АвтомАтическое определение грАниц словА в русском тексте  с помощью комплексА лингвистических прАвил"
[]
АвтомАтическое определение грАниц словА в русском 

loaded to catalogue
SELECT id FROM article WHERE title="РАЗРАБОТКА МЕТОДА ПОИСКА КЛЮЧЕВЫХ СЛОВ ДЛЯ ЗАДАЧИ ВИЗУАЛИЗАЦИИ НАУЧНЫХ СТАТЕЙ"
[]
РАЗРАБОТКА МЕТОДА ПОИСКА КЛЮЧЕВЫХ СЛОВ ДЛЯ ЗАДАЧИ ВИЗУАЛИЗАЦИИ НАУЧНЫХ СТАТЕЙ id =None
loaded to article
SELECT id FROM author WHERE  name = "Sandrikova Maria "
loaded to author
SELECT id FROM author WHERE  name = "Zhuravlev Aleksei "
loaded to author
SELECT id FROM author WHERE  name = "Sinevich Valeria "
loaded to author
SELECT id FROM author WHERE  name = "Korbut Daniel "
loaded to author
SELECT id FROM author WHERE  name = "Iglina Alexandra "
loaded to author
SELECT id FROM author WHERE  name = "Fedotov Semen "
loaded to author
SELECT id FROM conference WHERE  year = 2017 AND conference = "Dialog";
SELECT id FROM catalogue WHERE  author_id = 1541 AND article_id = 1020 AND conference_id = 18;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 1542 AND article_id = 1020 AND conference_id = 18;
loaded to catalogue
SELECT id FROM catalog

Загрузка статей AIST:

In [56]:
with open("aist.pickle", 'rb') as f:
    dialog = pickle.load(f)
    print(len(dialog), 'articles')
    c.load_to_aist(dialog)

59 articles
SELECT id FROM article WHERE title="Интеграция информационных систем с применением семантических технологий "
[(1,)]
Интеграция информационных систем с применением семантических технологий  id =1
SELECT id FROM author WHERE  name = "Сергей Горшков "
SELECT id FROM conference WHERE  year = 2013 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 1 AND article_id = 1 AND conference_id = 28;
SELECT id FROM article WHERE title="Генерация сниппетов в поисковых системах как задача автоматического квазиреферирования "
[(2,)]
Генерация сниппетов в поисковых системах как задача автоматического квазиреферирования  id =2
SELECT id FROM author WHERE  name = "Лиана Ермакова "
SELECT id FROM conference WHERE  year = 2013 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 2 AND article_id = 2 AND conference_id = 28;
SELECT id FROM article WHERE title=" Использование ресурсов Интернета для построения таксономии  "
[(3,)]

Использование ресурсов Интернет

SELECT id FROM author WHERE  name = "Vera Danilova"
SELECT id FROM author WHERE  name = "Artem Egorov"
SELECT id FROM conference WHERE  year = 2014 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 42 AND article_id = 21 AND conference_id = 29;
SELECT id FROM catalogue WHERE  author_id = 43 AND article_id = 21 AND conference_id = 29;
SELECT id FROM catalogue WHERE  author_id = 44 AND article_id = 21 AND conference_id = 29;
SELECT id FROM article WHERE title=" Analysis of Twitter Users’ Mood for Prediction of Gold and Silver Prices in the Stock Market"
[(22,)]

Analysis of Twitter Users’ Mood for Prediction of Gold and Silver Prices in the Stock Market id =22
SELECT id FROM author WHERE  name = "Alexander Porshnev"
SELECT id FROM author WHERE  name = "Ilya Redkin"
SELECT id FROM conference WHERE  year = 2014 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 45 AND article_id = 22 AND conference_id = 29;
SELECT id FROM catalogue WHERE  author_id = 46

SELECT id FROM catalogue WHERE  author_id = 36 AND article_id = 38 AND conference_id = 32;
SELECT id FROM catalogue WHERE  author_id = 78 AND article_id = 38 AND conference_id = 32;
SELECT id FROM catalogue WHERE  author_id = 79 AND article_id = 38 AND conference_id = 32;
SELECT id FROM article WHERE title="Automated Detection of Non-Relevant Posts on the Russian Imageboard “2ch”: Importance of the Choice of Word Representations"
[(39,)]
Automated Detection of Non-Relevant Posts on the Russian Imageboard “2ch”: Importance of the Choice of Word Representations id =39
SELECT id FROM author WHERE  name = "Amir Bakarov"
SELECT id FROM conference WHERE  year = 2017 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 80 AND article_id = 39 AND conference_id = 32;
SELECT id FROM article WHERE title="Fighting with the Sparsity of Synonymy Dictionaries for Automatic Synset Induction"
[(40,)]
Fighting with the Sparsity of Synonymy Dictionaries for Automatic Synset Induction id =

SELECT id FROM author WHERE  name = "Galina Lavrentyeva"
SELECT id FROM author WHERE  name = "Sergey Novoselov"
SELECT id FROM author WHERE  name = "Konstantin Simonchik"
SELECT id FROM conference WHERE  year = 2016 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 107 AND article_id = 53 AND conference_id = 31;
SELECT id FROM catalogue WHERE  author_id = 108 AND article_id = 53 AND conference_id = 31;
SELECT id FROM catalogue WHERE  author_id = 109 AND article_id = 53 AND conference_id = 31;
SELECT id FROM article WHERE title="Combining Knowledge and CRF-Based Approach to Named Entity Recognition in Russian"
[(54,)]
Combining Knowledge and CRF-Based Approach to Named Entity Recognition in Russian id =54
SELECT id FROM author WHERE  name = "V.A. Mozharova"
SELECT id FROM author WHERE  name = "N.V. Loukachevitch"
SELECT id FROM conference WHERE  year = 2016 AND conference = "AIST";
SELECT id FROM catalogue WHERE  author_id = 110 AND article_id = 54 AND conference_id

In [52]:
with open("russir.pickle", 'rb') as f:
    dialog = pickle.load(f)
    print(len(dialog), 'articles')
    c.load_to_russir(dialog)

29 articles
SELECT id FROM article WHERE title="Linguistic Approach to Suicide Detection "
[]
Linguistic Approach to Suicide Detection  id =None
loaded to article
SELECT id FROM author WHERE  name = "Ermakov Sergei"
loaded to author
SELECT id FROM author WHERE  name = "Ermakova Liana "
loaded to author
SELECT id FROM conference WHERE  year = 2013 AND conference = "RuSSIR";
SELECT id FROM catalogue WHERE  author_id = 121 AND article_id = 60 AND conference_id = 38;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 122 AND article_id = 60 AND conference_id = 38;
loaded to catalogue
SELECT id FROM article WHERE title="Ranking in keyphrase extraction problem: is it suitable to use statistics of words occurrences? "
[]
Ranking in keyphrase extraction problem: is it suitable to use statistics of words occurrences?  id =None
loaded to article
SELECT id FROM author WHERE  name = "Svetlana V. Popova"
loaded to author
SELECT id FROM author WHERE  name = "Ivan A. Khodyrev"
loaded to 

loaded to catalogue
SELECT id FROM article WHERE title="Languages of Russia: Using Social Networks to Collect Texts"
[]
Languages of Russia: Using Social Networks to Collect Texts id =None
loaded to article
SELECT id FROM author WHERE  name = "Irina Krylova"
loaded to author
SELECT id FROM author WHERE  name = "Boris Orekhov"
loaded to author
SELECT id FROM author WHERE  name = "Ekaterina Stepanova"
loaded to author
SELECT id FROM author WHERE  name = "Lyudmila Zaydelman"
loaded to author
SELECT id FROM conference WHERE  year = 2015 AND conference = "RuSSIR";
SELECT id FROM catalogue WHERE  author_id = 142 AND article_id = 72 AND conference_id = 40;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 143 AND article_id = 72 AND conference_id = 40;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 144 AND article_id = 72 AND conference_id = 40;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 145 AND article_id = 72 AND conference_id = 40;
loaded

loaded to author
SELECT id FROM author WHERE  name = "M. Atif Qureshi"
loaded to author
SELECT id FROM author WHERE  name = "Nasir Touheed"
loaded to author
SELECT id FROM conference WHERE  year = 2011 AND conference = "RuSSIR";
SELECT id FROM catalogue WHERE  author_id = 163 AND article_id = 85 AND conference_id = 36;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 164 AND article_id = 85 AND conference_id = 36;
loaded to catalogue
SELECT id FROM catalogue WHERE  author_id = 165 AND article_id = 85 AND conference_id = 36;
loaded to catalogue
SELECT id FROM article WHERE title="narrow-DomaIn Short textS CluSterInG alGorIthm"
[]
narrow-DomaIn Short textS CluSterInG alGorIthm id =None
loaded to article
SELECT id FROM author WHERE  name = "Svetlana V. Popova"
SELECT id FROM author WHERE  name = "Ivan A. Khodyrev"
SELECT id FROM conference WHERE  year = 2011 AND conference = "RuSSIR";
SELECT id FROM catalogue WHERE  author_id = 123 AND article_id = 86 AND conference_id = 36

Поиск по имени (с использованием транслитерации):

In [9]:
print(c.select_articles_of_author('Кутузов'))

[(' Оценка сниппетов в поиске Mail.ru: корреляция автоматических и ассессорских оценок',), ('\x0cWebVectors: A Toolkit for Building Web Interfaces for Vector Semantic Models',), ('Корпус несовершенных переводов КаК инструмент для переводоведчесКих исследований',), ('ПОВЫШЕНИЕ КАЧЕСТВА ВЫРАВНИВАНИЯ ПО ПРЕДЛОЖЕНИЯМ ДЛЯ ПАРАЛЛЕЛЬНОГО АНГЛО-РУССКОГО КОРПУСА ПРИ ПОМОЩИ ЧАСТЕРЕЧНОЙ РАЗМЕТКИ (INCREASING SENTENCE ALIGNMENT QUALITY IN PARALLEL ENGLISH-RUSSIAN CORPUS THROUGH THE USE OF PART-OF-SPEECH TAGGING)',), ('The ImpacT of morphology processIng QualITy on auTomaTed anaphora resoluTIon  for russIan',), ('На входе тексты, На выходе смысл: НейроННые языковые модели для задач семаНтической близости (На материале русского языка)',), ('ArbitrAriness of Linguistic sign Questioned: correLAtion between word form And meAning in russiAn',), ('Evaluation tracks on Plagiarism DEtEction algorithms for thE russian languagE',)]


Поиск по конференции:

In [10]:
print(c.select_articles_from_conference('Dialog', 2010))

[('Устное дистантное общение: о некоторых текстовых особенностях диалогов по мобильному телефону (Distant oral communication: some text characteristics of mobile phone dialogues)',), ('Обучение классификаторов на основе выделения фрагментов (Classiﬁ er training by passage recognition)',), ('Топологический аспект многозначности идиом (Topological aspect of polysemy in semantics  of idioms)',), ('О распознавании жестов языка глухих (About recognition of sign language gestures)',), ('Автоматический поиск и классификация однословных терминов в корпусе предметной области с использованием  логарифмической меры сходства с неспециализированным корпусом (Automatic detection and classiﬁ cation of single-word terms in a speciﬁ c domain corpus  using log likelihood similarity with general purpose corpus)',), ('Такая девочка-девочка. Семантика редупликации существительных в русской разговорной речи и языке интернета (Such a girl-girl. Semantics of noun reduplication  in colloquial russian and the i

Поиск по аффилиации (с использованием транслитерации):

In [105]:
print(c.select_article_by_affiliation('РАН'))

SELECT DISTINCT article.title FROM catalogue JOIN author JOIN article ON catalogue.author_id=author.id AND catalogue.article_id=article.id WHERE author.affiliation LIKE '%РАН%' OR author.affiliation LIKE '%RAN%'
[('Relational Machine Learning Author Disambiguation',), ('О ПРОЕКТЕ АКТИВНОГО СЛОВАРЯ (АС) РУССКОГО ЯЗЫКА (ON A PROJECT OF A PRODUCTION DICTIONARY OF RUSSIAN)',), ('КОМПЛЕКСНАЯ ТЕХНОЛОГИЯ АВТОМАТИЧЕСКОЙ КЛАССИФИКАЦИИ ТЕКСТОВ (COMPLEX TECHNOLOGY OF AUTOMATIC TEXT CLASSIFICATION)',), ('КОРПУС ЗВУЧАЩЕЙ РУССКОЙ РЕЧИ В СОСТАВЕ НАЦИОНАЛЬНОГО КОРПУСА РУССКОГО ЯЗЫКА. ПРОЕКТ* (CORPUS OF ORAL RUSSIAN IN THE FRAMEWORK OF RUSSIAN NATIONAL CORPUS. CONSTRUCTION PROJECT)',), ('КОНСТРУКЦИЯ С ТВОРИТЕЛЬНЫМ ФОРМЫ «X Y-ОМ» (DESCRIBING SHAPE: INSTRUMENTAL CONSTRUCTION «X Y-ОМ»)',), ('СИНТАКСИЧЕСКИЙ АНАЛИЗАТОР ЛИНГВИСТИЧЕСКОГО ПРОЦЕССОРА ЭТАП-3: ЭКСПЕРИМЕНТЫ ПО РАНЖИРОВАНИЮ СИНТАКСИЧЕСКИХ ГИПОТЕЗ (THE PARSER OF ETAP-3 LINGUISTIC PROCESSOR: EXPERIMENTS ON RANKING SYNTACTIC HYPOTHESES)',), ('ТЕКСТЫ А

Поле для рисеча:

In [93]:
c.cursor.execute('SELECT affiliation FROM author')
a = c.cursor.fetchall()
print(a)

[('Stel – Computer Systems Ltd. Moscow, Russian Federation ',), ('Stel – Computer Systems Ltd. Moscow, Russian Federation ',), ('Stel – Computer Systems Ltd. Moscow, Russian Federation ',), ('Stel – Computer Systems Ltd. Moscow, Russian Federation ',), ('Stel – Computer Systems Ltd. Moscow, Russian Federation ',), ('The Academic University Saint-Petersburg, Russia',), ('The Academic University Saint-Petersburg, Russia',), ('Saint-Petersburg, Russia',), ('Kazan (Volga region) Federal University; Kazan, Russian Federation',), ('Kazan (Volga region) Federal University; Kazan, Russian Federation',), ('Kazan (Volga region) Federal University; Kazan, Russian Federation',), ('Saint Petersburg State University; Saint Petersburg, Russia',), ('Saint Petersburg State University; Saint Petersburg, Russia',), ('Saint Petersburg State University; Saint Petersburg, Russia',), ('ORT Braude College, Karmiel, Israel',), ('Odnoklassiniki LTD (ok.ru), St.-Petersburg, Russia',), ('Petrozavodsk State Univer